# PyCity Schools Analysis

* It's expected that schools with bigger budgets perform better because they can afford more resources and therefore, attract better teachers. That may be the case in some schools, but it's not guaranteed. The data shows that those schools that have a per student budget of \$614 or less performed better than those that spent between \$615 and \$675 per student. Although the average math and reading scores are not significantly better, the % of students with passing grades for math and reading is. 
* I expected to see the average math and reading scores vary by grade, but although they vary by school, they stay pretty consistent, grade by grade. This data point is interesting, but not significant.
* Another assumption most people make is that smaller schools perform better. This data actually backs up that assumption.  The passing rates for schools with less than 2000 students is significantly better, even though the average math and reading scores are similar. The 4 of the top 5 performing schools have fewer than 2000 students, whereas the bottom 5 performing schools have more than 2000 students. All 5 of the top 5 schools are charter and all 5 of the bottom 5 schools are district. It's hard to say with this current analysis whether charter schools perform better because they're a charter school or because they have fewer students. 

In [1]:
import pandas as pd

In [2]:
# Read in files and create data frames
students_csv = "raw_data/students_complete.csv"
students_df = pd.read_csv(students_csv)

schools_csv = "raw_data/schools_complete.csv"
schools_df = pd.read_csv(schools_csv)
schools_df = schools_df.rename(columns={"name": "school"})

In [3]:
# Determine totals for the district
total_schools = schools_df["school"].count()
total_students = schools_df["size"].sum()
total_budget = schools_df["budget"].sum()

In [4]:
# Determine average math and reading scores for the district
average_math_score = students_df["math_score"].mean()
average_reading_score = students_df["reading_score"].mean()

In [5]:
# Determine the number of students that received a score of 70 or greater in math for the whole district
passed_math_df = students_df["math_score"] >= 70  #gives all rows, true or false whether they are >= 70
passed_math_only = students_df.loc[passed_math_df, "math_score"].count() # gives a count of all the names that scored >= 70
passed_math_percent = passed_math_only / total_students * 100

In [6]:
# Determine the number of students that received a score of 70 or greater in reading for the whole district
passed_reading_df = students_df["reading_score"] >= 70  #gives all rows, true or false whether they are >= 70
passed_reading_only = students_df.loc[passed_reading_df, "reading_score"].count() # gives a count of all the names that scored >= 70
passed_reading_percent = passed_reading_only / total_students * 100

In [7]:
# Determine the overall passing rate in math and reading for the district
overall_passing_rate = (passed_math_percent + passed_reading_percent) / 2

# District Summary

In [8]:
district_summary = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students], \
                                 "Total Budget": [total_budget], "Average Math Score": [average_math_score], \
                                  "Average Reading Score":[average_reading_score], "% Passing Math": [passed_math_percent],
                                  "% Passing Reading": [passed_reading_percent], "Overall Passing Rate": [overall_passing_rate]})

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary = district_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score", \
                                     "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"]]
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


=======================================================================================================

In [9]:
# Format school file to the columns needed in final report
by_schools_df = schools_df.rename(columns={"type": "School Type", "size": "Total Students", "budget": "Total School Budget"})
by_schools_df["Per Student Budget"] = by_schools_df["Total School Budget"] / by_schools_df["Total Students"]
by_schools_df = by_schools_df.groupby("school").max()

In [10]:
# Math scores >= 70 by school
by_school_passed_math_only = students_df.loc[passed_math_df, ["school", "math_score"]]
by_school_passed_math_only = by_school_passed_math_only.groupby("school").count()

In [11]:
# Reading scores >= 70 by school
by_school_passed_reading_only = students_df.loc[passed_reading_df, ["school", "reading_score"]]
by_school_passed_reading_only = by_school_passed_reading_only.groupby("school").count()

In [12]:
# Average math and reading scores by school
average_scores = students_df.groupby("school")[["reading_score", "math_score"]].mean()
average_scores = average_scores.rename(columns={"math_score": "Average Math Score", "reading_score": "Average Reading Score"})
average_scores = average_scores[["Average Math Score", "Average Reading Score"]]
average_scores["% Passing Math"] = by_school_passed_math_only["math_score"] / by_schools_df["Total Students"] * 100
average_scores["% Passing Reading"] = by_school_passed_reading_only["reading_score"] / by_schools_df["Total Students"] * 100
average_scores["% Overall Passing Rate"] = (average_scores["% Passing Math"] + average_scores["% Passing Reading"]) / 2

# School Summary

In [13]:
school_summary_df = pd.merge(by_schools_df, average_scores, left_index=True, right_index=True)
display_summary_df = school_summary_df.drop("School ID", axis=1)
display_summary_df["Total School Budget"] = display_summary_df["Total School Budget"].map("${:,.2f}".format)
display_summary_df["Per Student Budget"] = display_summary_df["Per Student Budget"].map("${:,.2f}".format)
del display_summary_df.index.name
display_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


# Top Performing Schools (By Passing Rate)

In [29]:
top_5_schools = display_summary_df.sort_values(["% Overall Passing Rate"], ascending=False)
top_5_schools = top_5_schools.iloc[0:5, :]
del top_5_schools.index.name
top_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Passing Rate)

In [30]:
bottom_5_schools = display_summary_df.sort_values(["% Overall Passing Rate"])
bottom_5_schools = bottom_5_schools.iloc[0:5, :]
del bottom_5_schools.index.name
bottom_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


# Average Math Scores by Grade

In [16]:
ninth_grade = students_df.loc[(students_df["grade"] == "9th"), :]
ninth_grade_avg_math = ninth_grade.groupby("school")["math_score"].mean()
tenth_grade = students_df.loc[(students_df["grade"] == "10th"), :]
tenth_grade_avg_math = tenth_grade.groupby("school")["math_score"].mean()
eleventh_grade = students_df.loc[(students_df["grade"] == "11th"), :]
eleventh_grade_avg_math = eleventh_grade.groupby("school")["math_score"].mean()
twelveth_grade = students_df.loc[(students_df["grade"] == "12th"), :]
twelveth_grade_avg_math = twelveth_grade.groupby("school")["math_score"].mean()

In [17]:
math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_avg_math, "10th": tenth_grade_avg_math, \
                                     "11th": eleventh_grade_avg_math, "12th": twelveth_grade_avg_math})
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
del math_scores_by_grade.index.name
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Average Reading Scores by Grade

In [18]:
ninth_grade_avg_reading = ninth_grade.groupby("school")["reading_score"].mean()
tenth_grade_avg_reading = tenth_grade.groupby("school")["reading_score"].mean()
eleventh_grade_avg_reading = eleventh_grade.groupby("school")["reading_score"].mean()
twelveth_grade_avg_reading = twelveth_grade.groupby("school")["reading_score"].mean()

In [19]:
reading_scores_by_grade = pd.DataFrame({"9th": ninth_grade_avg_reading, "10th": tenth_grade_avg_reading, \
                                        "11th": eleventh_grade_avg_reading, "12th": twelveth_grade_avg_reading})
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
del reading_scores_by_grade.index.name
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Average Scores by School Spending

In [20]:
# Create bins for school budgets
bins = [0, 585, 615, 645, 675]

# Create labels for the bins
spending = ["<\$585", "\$585 - $614", "\$615 - $644", "\$645 - $675"]

In [21]:
# Slice the data and place it into bins
rank = pd.cut(school_summary_df["Per Student Budget"], bins, labels=spending)

In [22]:
# Add column to main dataframe
school_summary_df["Spending Ranges (Per Student)"] = rank

In [23]:
spending_groups = school_summary_df.groupby("Spending Ranges (Per Student)").mean()
spending_groups = spending_groups[["Average Math Score", "Average Reading Score", "% Passing Math", \
                                   "% Passing Reading", "% Overall Passing Rate"]]
spending_groups

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<\$585,83.455399,83.933814,93.460096,96.610877,95.035486
\$585 - $614,83.599686,83.885211,94.230858,95.900287,95.065572
\$615 - $644,79.079225,81.891436,75.668212,86.106569,80.887391
\$645 - $675,76.997210,81.027843,66.164813,81.133951,73.649382


# Average Scores by School Size

In [24]:
# Create bins for school size
size_bins = [0, 1000, 2000, 5000]

# Create labels for the bins
size_buckets = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [25]:
# Slice the data and place it into bins
size = pd.cut(school_summary_df["Total Students"], size_bins, labels=size_buckets)

In [26]:
# Add column to main dataframe
school_summary_df["School Size"] = size

In [27]:
size_groups = school_summary_df.groupby("School Size").mean()
size_groups = size_groups[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",\
                           "% Overall Passing Rate"]]
size_groups

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


# Average Scores by School Type

In [28]:
school_type = school_summary_df.groupby("School Type").mean()
school_type = school_type[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",\
                           "% Overall Passing Rate"]]
school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
